# TensorFlow

## Module 04: Core Learning Algorithms: Categorization
https://www.freecodecamp.org/learn/machine-learning-with-python/tensorflow/core-learning-algorithms

freecodecamp.com

>[TensorFlow](#scrollTo=79GaUqHIT3OC)

>>[Module 04: Core Learning Algorithms: Categorization](#scrollTo=79GaUqHIT3OC)

>>>[Categorization](#scrollTo=3FOT-AYdU9S5)

>>>[Data](#scrollTo=_DvxuHHjixV0)

>>[Resources](#scrollTo=nSpipOqGclEm)



In this module we'll learn about 4 main algorithms:
1. Linear regression
2. Classification
3. Clustering
4. Hidden Markov Models

### 2. Categorization
Categorization is used to categorize. :)

In [ ]:
!pip install -q sklearn

In [ ]:
%tensorflow_version 2.x # this line is not required unless you're in a notebook

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import tensorflow as tf

### Data

The sample program in this document builds and tests a model that classifies Iris flowers into three different species based on the size of their sepals and petals.

This dataset has 4 categories of flowers:
- Setosa
- Versicolor
- Virginica

You will train a model using the Iris data set. The Iris data set contains four features and one label. The four features identify the following botanical characteristics of individual Iris flowers:

- sepal length
- sepal width
- petal length
- petal width

In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [ ]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

## Resources

- [X] [Build a linear model with Estimators](https://www.tensorflow.org/tutorials/estimator/premade)